# Tarea 2

Realizado por Catalina Morales Rojas y Vicente Vega Toro

# Problema 1

Para aumentar su competitividad, una compañía desea optimizar la manufactura de sus
productos más vendidos. La principal actividad de la compañía es la producción de tarjetas
electrónicas. La compañía, también produce los componentes para el ensamblado de las tarjetas.
Una buena planificación de la producción de los componentes constituye, por tanto, un factor de
éxito decisivo para la empresa. La demanda de los componentes en este caso es interna, por lo
tanto, fácil de proyectar.
Se quiere planificar para los próximos tres meses la producción de cuatro componentes
referenciados como AX22-M1, AX22-M2, CX32-N1 y CX32-N2. La demanda mensual para cada tipo
de componente se muestra en la siguiente tabla.



In [1]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")

using JuMP
using GLPK

modelo = Model(with_optimizer(GLPK.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

Las variables de decisión se definen como 

$$ X_{i,j}: unidades\ de\ componentes\ tipo\ i\ producidas\ en\ el\ mes\ j $$ 

con $i$ = 1,2,3,4 y $j$ = 1,2,3

$$ Y_{i,j}: unidades\ de\ componentes\ tipo\ i\ almacenadas\ al\ final\ del\ mes\ j$$

con $i$ = 1,2,3,4 y $j$ = 1,2,3

Se definen los datos utilizados para la definición de la función objetivo y las restricciones, donde

$ P $: es el costo (U.M.) de producción de los componentes.  
$ A $: es el costo (U.M.) de almacenaje de los componentes.   
$ D $: es la demanda de las unidades para los componentes.   
$ T $: Tiempo máquina para los componentes.

La función objetivo está definida como 

Minimizar $$ 20(X_{11}+X_{12}+X_{13}) + 25(X_{21} + X_{22} + X_{23}) + 10(X_{31}+X_{32}+X_{33}) + 15(X_{41}+X_{41}+X_{43}) + 0.4(I_{11}+I_{12}) + 0.5(I_{21} + I_{22}) + 0.3(I_{31} + I_{32}) + 0.3(I_{41} + I_{42}) $$

In [2]:
# Variables de decisión 
@variable(modelo,x[1:4,1:3] >= 0)
@variable(modelo,y[1:4,1:2] >= 0)

# Parámetros de la función
P = [20;25;10;15]
A = [0.4;0.5;0.3;0.3]

D = [1500 3000 2000;
     1300 800 800;
     2200 1500 2900;
     1400 1600 1500]

T = [0.2; 0.22; 0.24; 0.18]

# Función objetivo a minimizar
@objective(modelo, Min, sum(P[i]*x[i,k] for i in 1:4, k in 1:3) + sum(A[i]*y[i,k] for i in 1:4, k in 1:2))

20 x[1,1] + 20 x[1,2] + 20 x[1,3] + 25 x[2,1] + 25 x[2,2] + 25 x[2,3] + 10 x[3,1] + 10 x[3,2] + 10 x[3,3] + 15 x[4,1] + 15 x[4,2] + 15 x[4,3] + 0.4 y[1,1] + 0.4 y[1,2] + 0.5 y[2,1] + 0.5 y[2,2] + 0.3 y[3,1] + 0.3 y[3,2] + 0.3 y[4,1] + 0.3 y[4,2]

Para las restricciones se deben cumplir lo siguiente: 

1. El tiempo disponible no tiene que ser superado por el tiempo total de máquina que ha sido utilizado. 

$$ 0.2X_{11} + 0.22X_{21} + 0.24X_{31} + 0.18X_{41} \leq 2000  $$
$$ 0.2X_{12} + 0.22X_{22} + 0.24X_{32} + 0.18X_{42} \leq 2000  $$
$$ 0.2X_{13} + 0.22X_{23} + 0.24X_{33} + 0.18X_{43} \leq 2000  $$

2. Para cada producto se debe cumplir que el inventario inicial mas la producción menos el inventario final debe ser igual a la demanda. 

$$10 + X_{11} - I_{11} = 1500$$
$$I_{11} + X_{12} - I_{12} = 3000$$
$$I_{12} + X_{13} - 50 = 2000$$

$$X_{21} - I_{21} = 1300$$
$$I_{21} + X_{22} - I_{22} = 800$$
$$I_{22} + X_{23} - 10 = 800$$

$$X_{31} - I_{31} = 2200$$
$$I_{31} + X_{32} - I_{32} = 1500$$
$$I_{32} + X_{33} - 10 = 2900$$

$$X_{41} - I_{41} = 1400$$
$$I_{41} + X_{42} - I_{42} = 1600$$
$$I_{42} + X_{43} - 10 = 1500$$

In [3]:
for i in 1:3
    @constraint(modelo,sum(T[j]*x[j,i] for j in 1:4) <= 2000)
end

for i in 1:3
    @constraint(modelo,x[i+1,1] - y[i+1,1] == D[i+1,1] )
    @constraint(modelo,y[i+1,1] + x[i+1,2] - y[i+1,2] == D[i+1,2])
    @constraint(modelo,y[i+1,2] + x[i+1,3] - 10 == D[i+1,3])
end

@constraint(modelo,10+x[1,1]-y[1,1] == D[1,1])
@constraint(modelo,y[1,1] + x[1,2] - y[1,2] == D[1,2])
@constraint(modelo,y[1,2] + x[1,3] - 50 == D[1,3])

x[1,3] + y[1,2] = 2050.0

In [4]:
optimize!(modelo)
objective_value(modelo)

337300.0

Finalmente, se obtienen el valor para cada una de las variables definidas en el modelo.

Donde primero se tiene el valor que se debe producir de cada componente en un mes determinado.


In [5]:
for i in 1:4
    for j in 1:3
        println("Variable X",i,j," = ", value(x[i,j]))
    end
end

Variable X11 = 1490.0
Variable X12 = 3000.0
Variable X13 = 2050.0
Variable X21 = 1300.0
Variable X22 = 800.0
Variable X23 = 810.0
Variable X31 = 2200.0
Variable X32 = 1500.0
Variable X33 = 2910.0
Variable X41 = 1400.0
Variable X42 = 1600.0
Variable X43 = 1510.0


Por otro lado, se muestran la cantidad de componentes que estarán almacenados en un mes determinado, dando a entender que en ningún mes se estará guardando componentes.

In [6]:
for i in 1:4
    for j in 1:2
        println("Variable Y",i,j," = ", value(y[i,j]))
    end
end

Variable Y11 = 0.0
Variable Y12 = 0.0
Variable Y21 = 0.0
Variable Y22 = 0.0
Variable Y31 = 0.0
Variable Y32 = 0.0
Variable Y41 = 0.0
Variable Y42 = 0.0


# Problema 2

Se está prospectado una mina de uranio a cielo abierto. Sobre la base de los resultados de
algunas perforaciones de prueba, la mina se ha subdividido en unidades de explotación
denominadas bloques. Las excavaciones se deben realizar en forma aterrazada, con el fin de
permitir la circulación de los camiones. El yacimiento se extiende de este a oeste, viéndose
limitada por un pueblo y por un conjunto montañoso. Considerando estas limitaciones, se han
identificado 18 bloques de 10.000 toneladas distribuidos en tres niveles. Para extraer un bloque,
es necesario antes extraer tres bloques del nivel inmediatamente superior, siendo estos, el bloque
superior y los ubicados a la derecha y a la izquierda de este. Ejemplo: Para extraer el bloque 16,
antes se deben extraer los bloques 10, 11 y 12. 

Resolución:
Variables de decisión: 
    $x_{i}$ = puede tomar valores 0 o 1, donde 0 significa que no se extrae el bloque y 1 que si
    $i = 1,2,3,..., 18$

Función objetivo: 
    z: max $\sum_{i=1}^{18} (v_{i} - c_{i})*x_{i}$

La función objetivo corresponde a la suma del beneficio, es decir, el precio de venta del bloque menos el costo que tiene extraerlo, por x, el cual indica si el bloque se extrae o no. 

Restricciones:
    $x_{9} - x_{1} \leq 0$,
    $x_{9} - x_{2} \leq 0$,
    $x_{9} - x_{3} \leq 0$,
    $x_{10} - x_{2} \leq 0$,
    $x_{10} - x_{3} \leq 0$,
    $x_{10} - x_{4} \leq 0$,
    $x_{11} - x_{3} \leq 0$,
    $x_{11} - x_{4} \leq 0$,
    $x_{11} - x_{5} \leq 0$,
    $x_{12} - x_{4} \leq 0$,
    $x_{12} - x_{5} \leq 0$,
    $x_{12} - x_{6} \leq 0$,
    $x_{13} - x_{5} \leq 0$,
    $x_{13} - x_{6} \leq 0$,
    $x_{13} - x_{7} \leq 0$,
    $x_{14} - x_{6} \leq 0$,
    $x_{14} - x_{7} \leq 0$,
    $x_{14} - x_{8} \leq 0$,
    $x_{15} - x_{9} \leq 0$,
    $x_{15} - x_{10} \leq 0$,
    $x_{15} - x_{11} \leq 0$,
    $x_{16} - x_{10} \leq 0$,
    $x_{16} - x_{11} \leq 0$,
    $x_{16} - x_{12} \leq 0$,
    $x_{17} - x_{11} \leq 0$,
    $x_{17} - x_{12} \leq 0$,
    $x_{17} - x_{13} \leq 0$,
    $x_{18} - x_{12} \leq 0$,
    $x_{18} - x_{13} \leq 0$,
    $x_{18} - x_{14} \leq 0$
    
    
Las restricciones contempla que ningún bloque se puede extraer si es que los tres bloques de nivel superior (el inmediatamente superior y el que se encuentra a su izquierda y derecha), se extraen primero. 
Por lo anterior es que los bloques del 1 al 8 no tienen restricciones. 

In [10]:
import Pkg # Importa el administrador de paquetes Pkg
Pkg.add("JuMP") # Instala el paquete JuMP
Pkg.add("GLPK") # Instala el paquete GLPK
using JuMP # Importa el modulo JuMP.
using GLPK # Importa el modulo GLPK para utilizar el solver.

# Se define como modeloProblema2 a la construcción del modelo. 
modeloProblema2 = Model(with_optimizer(GLPK.Optimizer))

#Se define un arreglo con los costos que tiene extraer cada uno de los bloques, los cuales son correlativos a la
#posición en que se encuentran, es decir, el primer elemento indica que el bloque 1 cuesta 1.000.000 extraer una 
#tonelada, el segundo es respecto al bloque 2, y así sucesivamente. 
c = [1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000,10000000,2000000,2000000,2000000,2000000,10000000,10000000,10000000,3000000,10000000]

#También se define un arreglo que indica a cuanto se vende cada bloque, los cuales también son correlativos a la 
#posición en la que se encuentran. 
v = [2000000,0,0,0,0,0,3000000,0,0,5000000,0,2000000,0,0,0,0,10000000,12000000]


#Luego se define que el problema tendrá 18 variables de decisión, a las que se le asigna el nombre "x" y que son
#de tipo binaria, es decir, solo pueden tomar valores 0 o 1. Estas variables representan si el bloque se sacará o no
@variable(modeloProblema2, x[1:18], Bin)

#Se debe modelar con objetive la función objetivo, a la cual se le indica que es una maximización. Se le entrega el 
#arreglo de venta, costos y las variables de decisión. 
@objective(modeloProblema2, Max, sum((v[i]-c[i])*x[i] for i in 1:18))

#A continuación se definen con constraint las restricciones que tiene el modelo. Como ya se indicó que las variables de
#decisión solo pueden tomar valores 0 o 1, no es necesario volver a especificarlo en las restricciones. 
@constraint(modeloProblema2, x[9] - x[1] <= 0)
@constraint(modeloProblema2, x[9] - x[2] <= 0)
@constraint(modeloProblema2, x[9] - x[3] <= 0)
@constraint(modeloProblema2, x[10] - x[2] <= 0)
@constraint(modeloProblema2, x[10] - x[3] <= 0)
@constraint(modeloProblema2, x[10] - x[4] <= 0)
@constraint(modeloProblema2, x[11] - x[3] <= 0)
@constraint(modeloProblema2, x[11] - x[4] <= 0)
@constraint(modeloProblema2, x[11] - x[5] <= 0)
@constraint(modeloProblema2, x[12] - x[4] <= 0)
@constraint(modeloProblema2, x[12] - x[5] <= 0)
@constraint(modeloProblema2, x[12] - x[6] <= 0)
@constraint(modeloProblema2, x[13] - x[5] <= 0)
@constraint(modeloProblema2, x[13] - x[6] <= 0)
@constraint(modeloProblema2, x[13] - x[7] <= 0)
@constraint(modeloProblema2, x[14] - x[6] <= 0)
@constraint(modeloProblema2, x[14] - x[7] <= 0)
@constraint(modeloProblema2, x[14] - x[8] <= 0)
@constraint(modeloProblema2, x[15] - x[9] <= 0)
@constraint(modeloProblema2, x[15] - x[10] <= 0)
@constraint(modeloProblema2, x[15] - x[11] <= 0)
@constraint(modeloProblema2, x[16] - x[10] <= 0)
@constraint(modeloProblema2, x[16] - x[11] <= 0)
@constraint(modeloProblema2, x[16] - x[12] <= 0)
@constraint(modeloProblema2, x[17] - x[11] <= 0)
@constraint(modeloProblema2, x[17] - x[12] <= 0)
@constraint(modeloProblema2, x[17] - x[13] <= 0)
@constraint(modeloProblema2, x[18] - x[12] <= 0)
@constraint(modeloProblema2, x[18] - x[13] <= 0)
@constraint(modeloProblema2, x[18] - x[14] <= 0)

-x[14] + x[18] ≤ 0.0

In [6]:
#Por último, se resuelve el problema de optimización, el cuál entrega un valor de 4*10^6, lo que indica que esa es la
#ganancia que se obtiene al sacar los bloques. 
optimize!(modeloProblema2)
objective_value(modeloProblema2)

4.0e6

In [7]:
#Mediante el for también es posible obtener los valores que adquiere cada una de las variables de decisión, 
#las cuales si se encuentran en 1 indican que el bloque fue sacado y en 0 que no. En este caso, se sacaron 12 bloques.
for j in 1:18
    println("Variable X",j," = ",value(x[j]))
end

Variable X1 = 1.0
Variable X2 = 1.0
Variable X3 = 1.0
Variable X4 = 1.0
Variable X5 = 1.0
Variable X6 = 1.0
Variable X7 = 1.0
Variable X8 = 0.0
Variable X9 = 0.0
Variable X10 = 1.0
Variable X11 = 1.0
Variable X12 = 1.0
Variable X13 = 1.0
Variable X14 = 0.0
Variable X15 = 0.0
Variable X16 = 0.0
Variable X17 = 1.0
Variable X18 = 0.0
